# State-Machine States

### State Name : New Contract Creation (Yeni Sözleşme Oluşturma)

1. All of json copied without id's for process
2. Startdate is changed to currentdate

3. Changed startDate's endDates is also changed with active ones 

##### Pre-Execute Script

In [ ]:
# machine              : Currently executing machine instance
# state                : Currently executing state instance
# request_data         : Original trigger data
# context              : Machine stored context
# prev_state_response  : Previous executed state business logic response
# return
#     {"success": True, "goto": "STATE_CODE"}
#     {"success": False, "error": {"code": "...", "detail": "..."}}
def state_handler(machine, state, request_data, context, prev_state_response):
    # do workflow here
    return {"success": True, "goto": "END"}

##### Business Mapping

In [18]:
from datetime import datetime
import json
import numpy as np
def state_handler(machine, state, request_data, context, prev_state_response, current_state_response):
    # Not recursive 
    current_date = datetime.now().strftime('%Y%m%d') + '000000'
    newContractPower = next((attr for attr in context['attributes'] if attr['attribute']['attributeCode']=='NewContractPower'),None)
    
    changes_dict = {'ContractPower':newContractPower }
    
    copied_json = {}
    stack = [(copied_json, context)]

    # Copy context to copied_json without id
    while stack:
        current_copied, current_original = stack.pop()

        for key, value in current_original.items():
            if key != 'id':
                if isinstance(value, list):
                    new_list = []
                    for item in value:
                        if isinstance(item, dict):
                            new_item = {}
                            new_list.append(new_item)
                            stack.append((new_item, item))
                        else:
                            new_list.append(item)
                    current_copied[key] = new_list
                elif isinstance(value, dict):
                    new_dict = {}
                    current_copied[key] = new_dict
                    stack.append((new_dict, value))
                else:
                    current_copied[key] = value

    # Set dates for all values in copied_json
    for key, value in copied_json.items():
        if isinstance(value, list):
            for item in value:
                if isinstance(item, dict):
                    if item.get('value') is None and 'values' in item and item['values']:
                        item['values'][0]['startDate'] = current_date
                        item['values'][0]['endDate'] = 99991231235959
                    elif item.get('value') is not None:
                        item['startDate'] = current_date
                        item['endDate'] = 99991231235959
        elif isinstance(value, dict):
            if value.get('value') is None and 'values' in value and value['values']:
                value['values'][0]['startDate'] = current_date
                value['values'][0]['endDate'] = 99991231235959
            elif value.get('value') is not None:
                value['startDate'] = current_date
                value['endDate'] = 99991231235959

    # Update attribute values in context according to the dict above
    for code, new_value in changes_dict.items():
        for attr in copied_json.get('attributes', []):
            if attr['attribute']['attributeCode'] == code:
                if attr['value'] is None:
                    attr['values'][0]['value'] = new_value
                    attr['values'][0]['valueDesc'] = None
                else:
                    attr['values'][0]['valueDesc'] = None
                    attr['value'] = new_value


    return copied_json  

In [17]:
from datetime import datetime

def copy_json_without_ids(original_json):
    #recursive one
    current_date = datetime.now().strftime('%Y%m%d%H%M%S')
    
    copied_json = {}

    for key, value in original_json.items():
        if key != 'id':
            if isinstance(value, list):
                copied_json[key] = []
                for item in value:
                    if isinstance(item, dict):
                        copied_item = copy_json_without_ids(item)                    
                        if copied_item.get('value') is None and 'values' in copied_item and copied_item['values']:
                            copied_item['values'][0]['startDate'] = current_date
                            copied_item['values'][0]['endDate'] = 99991231235959  
                        elif copied_item.get('value') is not None:
                            copied_item['startDate'] = current_date
                            copied_item['endDate'] = 99991231235959  
                        copied_json[key].append(copied_item)
                    else:
                        copied_json[key].append(item)
            elif isinstance(value, dict):
                copied_json[key] = copy_json_without_ids(value)
            else:
                copied_json[key] = value

    return copied_json